In [ ]:
using Pkg; Pkg.activate(".")
using Revise
import Plots: @animate, plot3d, gif, @gif
import PyPlot
import PyPlot: subplot, title, xlabel, ylabel, ylim, xlim, plot
import LinearAlgebra: norm, dot
import Base
import SparseArrays: spdiagm, sprand, spzeros

include("Opt.jl")
import .Opt

In [ ]:
n = Int64(1e2)
f, grad! = Opt.hard_leastsquares_problem(n)

x = sprand(n, 0.5)
g = similar(x)
grad!(g,x)

@show f(x)
@show size(g)

In [ ]:
# pre-allocated
x0 = spzeros(n)
n_iterations = 10

Γf = zeros(n_iterations, n+1)
grad_norm = zeros(n_iterations)

function access_state(state)
    Γf[state.k,:] = [state.x..., state.f]
    grad_norm[state.k] = norm(state.g)
end

Opt.gradient_descent(x0, f, grad!;
    α = 0.001,
    n_iterations = n_iterations,
    access_state = access_state)

In [ ]:
subplot(211)
PyPlot.plot(1:n_iterations, Γf[:,end])
title("f(x)")
xlabel("k")

subplot(212)
PyPlot.plot(1:n_iterations, grad_norm)
title("|∇f(x)|")
xlabel("k")

In [ ]:
x0 = spzeros(n)
n_iterations = 1000
α₀ = 0.01
barzilaiborwein_type = 1
g_abstol = 1e-8
access_state(state) = nothing

x = copy(x0)
g = copy(x0); grad!(g, x)
u, v = zeros(size(x0)), zeros(size(x0))

s = Opt.GradientDescentBarzilaiBorweinState(x0)
grad!(s.g, x)

for k in 1:n_iterations
    
    s.k, s.f = k, f(x)
    @. s.x_prev = s.x
    @. s.x = x
    @. s.g_prev = s.g
    @. s.g = g

    if k == 1
        # should do line search
        s.α = α₀
    else
        u = s.x - s.x_prev
        v = s.g - s.g_prev
        s.α = (barzilaiborwein_type == 1) ?
            dot(u, v) / norm(v)^2 :
            norm(u)^2 / dot(u, v)
    end

    access_state(s)        

    x = s.x + s.α*(-s.g)
    grad!(g, x)

    if norm(s.g) <= g_abstol
        println("Terminate at k=$k: |∇f(x)| = $(norm(s.g)) <= $g_abstol")
        break
    end
end

In [ ]:
x0 = spzeros(n)
n_iterations = 10
fs = zeros(n_iterations)
grad_norm = zeros(n_iterations)
αs = zeros(n_iterations)

function access_state(state)
    fs[state.k] = state.f
    grad_norm[state.k] = norm(state.g)
    αs[state.k] = state.α
end

Opt.gradient_descent_barzilaiborwein(x0, f, grad!;
    n_iterations = n_iterations,
    access_state = access_state)

In [ ]:
N = 10
xs = 2:N

subplot(221)
PyPlot.plot(xs, fs[xs])
ylabel("f(x)")
xlabel("k")

subplot(222)
PyPlot.plot(xs, grad_norm[xs])
ylabel("|∇f(x)|")
xlabel("k")

subplot(223)
PyPlot.plot(xs, αs[xs])
ylabel("α")
xlabel("k")

In [ ]:
import Plots
import Plots: @animate, plot3d, gif

step = 1
s = collect(1:step:n_iterations)
fs = fs[s]

x = range(-2;stop=2,length=100)
y = range(-1;stop=3,length=100)
z = x .+ y

plt = Plots.plot(
    range(-2;stop=2,length=100),
    range(-1;stop=3,length=100),
    (x,y) -> rosenbrock([x y]),
    st=:contourf, camera=(-30,30))

plt = Plots.plot(
    1,
    xlim = (-2, 2),
    ylim = (-1, 3),
    zlim = (0, 2500),
    title = "Descent Methods",
    marker = 3)

anim = @animate for k=1:size(s, 1)
    push!(plt, Γfs[k,1], Γfs[k,2], Γfs[k])
end

Plots.gif(anim, "tmp.gif", fps = 15)